In [ ]:
import os
from raphtory import Graph

def load_graphs_lazy(graph_paths):
    """Generator function to lazily load graphs from files."""
    for path in graph_paths:
        if os.path.isfile(path):
            print(f"Loading graph from: {path}")
            yield Graph.load_from_file(path)
        else:
            print(f"File does not exist: {path}")
            yield None

graph_paths = [f"C:/ms_bincode/Graph_{i}" for i in range(1, 25)]

# Use the generator to iterate over graphs lazily
graphs_lazy = load_graphs_lazy(graph_paths)

In [ ]:
from raphtory import Graph
import numpy as np
import pickle

"""
Convertion of multiple graph binecode files into adjacency matrices
"""

def load_graphs_lazy(graph_paths):
    """Generator function to lazily load graphs from files."""
    for path in graph_paths:
        if os.path.isfile(path):
            print(f"Loading graph from: {path}")
            yield Graph.load_from_file(path)
        else:
            print(f"File does not exist: {path}")
            yield None

def transfor_into_adjacency_matrix(graph):
    node_index = {node_name: i for i, node_name in enumerate(list(graph.vertices.name))}
    adjacency_matrix = np.zeros((len(node_index), len(node_index)), dtype=int)
    for edge in graph.edges:
        src_index = node_index[edge.src.name]
        dst_index = node_index[edge.dst.name]
        adjacency_matrix[src_index, dst_index] = 1
    return adjacency_matrix

graph_paths = [f"C:/ms_bincode/Graph_{i}" for i in range(4, 5)]

# Use the generator to iterate over graphs lazily
graphs_lazy = load_graphs_lazy(graph_paths)

# Initialise a nested dictionary
A_dict = {}

# Use the generator to iterate over graphs lazily
for i, graph in enumerate(graphs_lazy):
    if graph is not None:
        # Implement a logic to divide the graph into one-minute segments.
        # For each minute segment, generate an adjacency matrix and store it.
        A_dict[f"g{i+1}"] = {}
        for windowed_graph in graph.rolling(window=60000):
            # Replace the following line with actual logic to get the specific minute's graph segment
            minute_graph_segment = windowed_graph  # Placeholder for actual segmentation logic
            adjacency_matrix = transfor_into_adjacency_matrix(minute_graph_segment)
            A_dict[f"g{i+1}"][str(windowed_graph)] = adjacency_matrix


# Pickle the nested dictionary
with open("graphs_adjacency_matrices.pkl", "wb") as f:
    pickle.dump(A_dict, f)

print("Nested dictionary pickled successfully.")

In [ ]:
from raphtory import Graph
import os
import numpy as np
import pickle

"""
Convertion of a single graph binecode file into an adjacency matrix
"""

def transfor_into_adjacency_matrix(graph):
    node_index = {node_name: i for i, node_name in enumerate(list(graph.vertices.name))}
    adjacency_matrix = np.zeros((len(node_index), len(node_index)), dtype=int)
    for edge in graph.edges:
        src_index = node_index[edge.src.name]
        dst_index = node_index[edge.dst.name]
        adjacency_matrix[src_index, dst_index] = 1
    return adjacency_matrix


# Specify the file path
binecode_path = "D:/Graph_4"

# Load the file
if os.path.isfile(binecode_path):
    g = Graph.load_from_file(binecode_path)
    print(g)
else:
    print(f"File does not exist: {binecode_path}")

# Initialise a nested dictionary
A_dict = {}

# For each minute segment, generate an adjacency matrix and store it.
A_dict["g"] = {}
for windowed_graph in g.rolling(window=60000):
    # Replace the following line with actual logic to get the specific minute's graph segment
    minute_graph_segment = windowed_graph  # Placeholder for actual segmentation logic
    adjacency_matrix = transfor_into_adjacency_matrix(minute_graph_segment)
    A_dict["g"][str(windowed_graph)] = adjacency_matrix


# Pickle the nested dictionary
with open("graphs_adjacency_matrices.pkl", "wb") as f:
    pickle.dump(A_dict, f)

print("Nested dictionary pickled successfully.")

In [1]:
from raphtory import Graph
import numpy as np
import pickle

# Incoporate lazy loading

# Identify all unique nodes
#print(list(g.vertices.name))
# [ms3123,ms323,...]

# Create a node index map
node_index = {node_name: i for i, node_name in enumerate(list(g.vertices.name))}
#print(node_index)
# [ms3123:0, ms323:1,...]

# Initialise adjacency matrix
adjacency_matrix = np.zeros((len(list(g.vertices.name)), len(list(g.vertices.name))), dtype=int)

# Populate adjacency matrix
for edge in g.edges:
    src_index = node_index[edge.src.name]
    dst_index = node_index[edge.dst.name]

    adjacency_matrix[src_index, dst_index] = 1

#print(adjacency_matrix)
#[[0 1 0 ... 0 0 0]
# [0 0 0 ... 0 0 0]
# [0 0 1 ... 0 0 0]
# ...
# [0 0 0 ... 0 0 0]
# [0 0 0 ... 0 0 0]
# [0 0 0 ... 0 0 0]]

A_dict = {'1': adjacency_matrix}
print(A_dict)

# Turn in into a nested dictionaries

# Such as:
# A_dict = {
#  "g1" : {
#       "1" : adjacency_matrix,
#       "2" : adjacency_matrix,
#           .
#           .
#           .
#   },
#   "g2" : {
#       "1" : adjacency_matrix,
#       "2" : adjacency_matrix,
#           .
#           .
#           .
#   },
#}

# Pickle the dictionary variable
with open("graphs_adjacency_matrices.pkl", "wb") as f:
    pickle.dump(A_dict, f)

NameError: name 'g' is not defined